In [1]:
# Load fMRI data
# You must first run 'python getdata.py' in the LatentSimilarity directory to get the data
# The data is from: https://openneuro.org/datasets/ds004144/versions/1.0.1
# We have 66 subjects, 33 of who have fibromyalgia and 33 of who are controls
# fMRI is upper triangle of 264x264 functional connectivity based on Power atlas

import pickle

fmriData = None

with open('../data/fmri-FC-slim.pkl', 'rb') as f:
    fmriData = pickle.load(f)
    
list(fmriData.keys())

['FC-slim', 'subjNum2IdxMap', 'subjIdx2NumMap', 'groupsNormalDiagMap']

In [2]:
# Package fMRI data into data matrix and response variables
# Important to have a balanced training set

import numpy as np

keys = list(fmriData['groupsNormalDiagMap'].keys())
y = [fmriData['groupsNormalDiagMap'][key] for key in keys]
y = np.array(y).astype('int')
x = [fmriData['FC-slim'][fmriData['subjNum2IdxMap'][key]] for key in keys]
x = np.stack(x)
print(x.shape)
print(y.shape)

(66, 34716)
(66,)


In [18]:
import sys

if '..' not in sys.path:
    sys.path.append('..')

from latsim import LatSim
from latsim import train_sim_ce
import torch 
import torch.nn as nn
import torch.nn.functional as F

aacc = []

for i in range(300):
    yy = torch.from_numpy(y).cuda()
    idcs = torch.randperm(66)
    yy = yy[idcs]
    yy = F.one_hot(yy).float()
    yytr = yy[:50]
    yyv = yy[50:60]
    yyt = yy[60:]
    xx = torch.from_numpy(x).float().cuda()
    xx = xx[idcs]
    xxtr = xx[:50]
    mu = 0#torch.mean(xxtr, dim=0, keepdims=True)
    sig = 1#torch.std(xxtr, dim=0, keepdims=True)
    xxtr = (xxtr-mu)/sig
    xxv = (xx[50:60]-mu)/sig
    xxt = (xx[60:]-mu)/sig
    sim = LatSim(xx.shape[1])
    train_sim_ce(sim, xxtr, yytr, 0.35, xv=xxv, yv=yyv, lr=2e-5, nepochs=100, pperiod=20, verbose=False)
    yhat = sim(xxtr, yytr, xxt)
    yhat = torch.argmax(yhat, dim=1)
    acc = torch.sum(yhat == torch.argmax(yyt, dim=1))/6
    print(acc)
    aacc.append(float(acc))
    
print(sum(aacc)/300)

tensor(0.8333, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(1., device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.8333, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.8333, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.8333, device='cuda:0')
tensor(0.1667, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.8333, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.1667, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.1667, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.8333, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.833

tensor(0.6667, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.8333, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.8333, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.1667, device='cuda:0')
tensor(0.1667, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0., device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.8333, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.8333, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.5000, device='cuda:0')
tensor(0.8333, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.1667, device='cuda:0')
tensor(0.8333, device='cuda:0')
tensor(0.3333, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.666

In [7]:
# Tune hyperparameters
# Look for get_default_distributions() in train.py to see the range of hyperparameters being tuned
# Hyperparameter tuning only supported for single-task models (multi-modal allowed)
# Use the LatSim class (in latsim.py) directly for multi-task models

import sys

sys.path.append('..')

from latsim.train import tune

# Make splits (regular cv may be unreliable)
splits = []
for _ in range(40):
    idcs = np.arange(66)
    np.random.shuffle(idcs)
    splits.append((idcs[:50],idcs[50:]))

best = tune(X, y, 'class', n_iter=50, cv=splits)

print('Complete')

Complete


In [8]:
# Print results

idcs = np.argsort(best.cv_results_['mean_test_score'])[::-1]
rng = idcs[:8]
print(rng)
print(best.cv_results_['mean_test_score'][rng])
print(best.cv_results_['std_test_score'][rng])

[ 1  4 22  6 11 39 21 20]
[0.675     0.675     0.665625  0.6546875 0.63125   0.6125    0.609375
 0.6046875]
[0.10752907 0.09601432 0.09222722 0.12182491 0.10625    0.13419319
 0.09757809 0.08195309]


In [9]:
# Print hyperparameters
print(best.cv_results_['param_standardize'][rng])
print(best.cv_results_['param_edp'][rng])
print(best.cv_results_['param_dp'][rng])
# print(best.cv_results_['param_wInit'][rng])
print(best.cv_results_['param_dim'][rng])
print(best.cv_results_['param_lr'][rng])
print(best.cv_results_['param_weight_decay'][rng])

[True True True True True True True True]
[0.2 0 0.2 0 0 0 0 0.2]
[0.2 0.5 0.2 0.2 0.5 0.5 0 0.2]
[1 10 2 2 10 10 2 2]
[0.1 0.1 0.01 0.01 0.01 0.001 0.0001 0.0001]
[0 0 0 0 0.0001 0.0001 0 0.0001]


In [10]:
# Validate hyperparameters
# LatSimEstimator has the sklearn interface (and so does not allow multi-task models)

import sys

sys.path.append('..')

from latsim.train import LatSimEstimator
import numpy as np
import random

acc = []

for split in range(50):
    params = LatSimEstimator.get_default_params()
    params['regOrClass'] = 'class'
    params['standardize'] = True
    params['edp'] = 0.2
    params['dp'] = 0.2
    params['wInit'] = 1e-4
    params['dim'] = 1
    params['lr'] = 1e-1
    params['weight_decay'] = 0
    sim = LatSimEstimator(**params)

    # Make a random split
    idcs = np.arange(66)
    np.random.shuffle(idcs)
    trainIdcs = idcs[:50]
    testIdcs = idcs[50:]
    Xtr = X[trainIdcs]
    Xt = X[testIdcs]
    ytr = y[trainIdcs]
    yt = y[testIdcs]

    sim.fit(Xtr, ytr)
    res = sim.predict(Xt)
    acc.append(np.mean(res == yt))
    print(acc[-1])

acc = np.array(acc)
print(np.mean(acc))
print(np.std(acc))


0.5
0.5
0.5
0.75
0.6875
0.75
0.75
0.5
0.75
0.6875
0.6875
0.75
0.625
0.625
0.625
0.6875
0.375
0.5625
0.5
0.3125
0.6875
0.75
0.6875
0.8125
0.5
0.375
0.75
0.625
0.6875
0.9375
0.625
0.6875
0.6875
0.75
0.625
0.5
0.5625
0.6875
0.625
0.625
0.9375
0.5625
0.625
0.875
0.5
0.5625
0.5
0.8125
0.8125
0.625
0.6425
0.13348689074212494
